# Setup: Import libraries and connect to Foursquare API

In [7]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
import folium # plotting library
import json
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

In [6]:
# Foursquare API credentials are stored on my google drive in a .py file.  Load them here.
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
libpath = '/content/drive/My Drive/Programming/Python/Coursera Data Science Capstone'
sys.path.append(libpath)
import fsa
drive.flush_and_unmount()
#print(fsa.CLIENT_ID+fsa.CLIENT_SECRET+fsa.ACCESS_TOKEN)
VERSION = '20180604'
LIMIT = 100

Mounted at /content/drive


# Question 1: Scrape wikipedia and clean up dataframe

In [21]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(table))[0] #create the df
df = df[df['Borough'] != 'Not assigned'] #get rid of not assigned boroughs
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood']) #borough but not neighborhood then neighborhood = borough
print(df.head())
print(df.shape)

  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


# Question 2: Add lat and long info